# <b>Generic Lifetime Annotations</b>
>**Lifetime is a construct of rust borrow checker that ensures that all borrow are valid.** <br>

Explicit lifetime annotation --> `<'a, GenericType>` or `&'a` var_name. <br>
Typically, we only need to specify lifetimes where we are passing references. <br>
In other cases, rust will infer the lifetime by itself, this is called lifetime elision.
<pre>
<ul><b>
    <li>&i32          a ref</li>
    <li>&'a i32       a ref with lifetime annotation</li>
    <li>&'a mut i32   a mutable ref with lifetime annotation</li>
</b></ul>
</pre>

> Below code fails, x does not live long enough. This happened because of lifetimes. x went out of scope, i.e it's lifetime ended but r had a reference to it. This will lead to dangling reference and therefore rust did not allow this.

In [8]:
{
    let r;                    //--------------+ 'a
    {                         //              |
        let x = 69;           //-+--'b        |    lifetimes of x and r visualized
        r = &x;               //              |
    }                         //-+---+        |
    println!("r: {}", r);     //--------------|
}

Error: `x` does not live long enough

In [9]:
{
    let x = 69;               //--------------+ 'a
    let r = &x;               //--------------+ 'b  In this case, their lifetimes start at their declaration
                              //              |     and end at the end of the program
    println!("r: {}", r);     //--------------|
}

r: 69


()

> 

In the following case, the return value depends on 2 variables. <br> It can return either x or y. But we dont know which one.

In [2]:
fn longest(x: &str, y: &str) -> &str {
    if x.len() > y.len() {
        x
    } else {
        y
    }
}

Error: missing lifetime specifier

If rust were to allow it, That would lead to dangling pointer errors in cases like the following:

In [ ]:
let string1 = String::from("abcd");
let result;
{
    let string2 = "xyz";
    result = longest(string1.as_str(), string2);
}
println!("The longest string is {}", result);

In the above case, string2 is in a seperate scope. <br>
longest returns a ref to string2 and stores it in result.<br>
when the scope ends, string2 is dropped. <br>
But result still store a ref to that location where string2 was once stored.<br>
This is a dangling pointer.

Writing the code with type annotations, the code becomes valid.

In [5]:
fn longest<'a>(x: &'a str, y: &'a str) -> &'a str {
    if x.len() > y.len() {
        x
    } else {
        y
    }
}

In the above function, x, y and return value, all 3 have same lifetime.<br>
This does not mean that x and y should be must have the same lifetime.<br>
It says, the return value will be available just before the point where either x or y goes out of scope.<br>
Or in other words,<br>
The result will be dropped as soon as either x or y is dropped.

In [16]:
{
    // now if we try to write this code;
    let s1 = String::from("abcd");
    let res;
    {
        let s2 = "xyz".to_string();
        res = longest(&s1, &s2);
    }
    println!("The longest string is {}", res);
    // we get an error because s2 is dropped before result is used
}

Error: `s2` does not live long enough

You cannot return reference to a variable thats created inside the function.<br>
the function returns the ref to x but x is dropped when function ends, this leads to a dangling pointer.

In [17]:
fn return_internal_reference<'a>() -> &'a i32 {
    let x = 69;
    &x
}

Error: cannot return reference to local variable `x`

Although we cannot return a ref, but is we really want to return some information, we will transfer the ownership of that variable to the caller scope.

>**Structs containing ref:** When we create structs we have just passed data to structs.<br>
If we want to save ref to some data in a struct, we have to specify lifetime.<br>
The below lifetime annotation says that:<br>
The struct instance will be dropped as soon as the variable ref stored in it is dropped.

In [18]:
struct ImportantExcerpt<'a> {
    part: &'a str,
}

## Lifetime Elision Rules
- Each parameter that is a reference get its own lifetime parameter.
- If there is exactly 1 input lifetime parameter, that lifetime is assigned to all output lifetime paramters.
- If there are multiple input lifetime parameters, but one of them is &self or &mut self, the lifetime of self is assigned to all output lifetime parameters.

If rust still cant infer what the lifetimes are, then only we have to manually specify the lifetimes.

In [19]:
impl<'a> ImportantExcerpt<'a> {
    fn return_part(&self, announcement: &str) -> &str {
        println!("Attention please: {}", announcement);
        self.part
    }
}

We didnt specify lifetimes in the above example because rust was able of infer it using the rules.<br>
First it gave a lifetime to both self and announcement.<br>
Then it checked for rule 2, fail.<br>
Then it checked for rule 3, &self was there, so it gave lifetime of self to return value.

## `'static` lifetime
Static lifetime tells the borrow checker that the variable lives as long as the program runs.

In [20]:
fn main(){
    let s: &'static str = "I have a static lifetime.";
}

# Example of Generic Lifetime Annotations

In [ ]:
fn longest_with_print<'a, T>(
    x: &'a str,s
    y: &'a str,
    ann: T,
) -> &'a str
    where T: std::fmt::Display
{
    println!("Announcement! {}", ann);
    if(x.len() > y.len()) {
        x
    } else {
        y
    }
}